In [2]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"G:\datasets\Underwater_Image\WHOI\archive\dataset_pm\training"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [10:01<00:00, 31.67s/it]

X_train shape: (2872, 32, 32, 3)
Y_train shape: (2872,)


In [3]:
import dask.dataframe as dd
import numpy as np
from tqdm import tqdm

# Define the paths to the CSV files
file_paths = [
    "G:/image_features.csv",
    "G:/image_features1.csv",
    "G:/image_features2.csv",
    "G:/image_features3.csv"
]

# Load the CSV files into Dask DataFrames
dfs = [dd.read_csv(file_path) for file_path in file_paths]

# Function to get 25,000 random indexes and retrieve the full rows
def get_random_rows(df, n=25000):
    # Convert the Dask DataFrame to a Pandas DataFrame
    pandas_df = df.compute()
    
    # Get the total number of rows in the DataFrame
    total_rows = len(pandas_df)
    
    # Generate 25,000 random indexes
    random_indexes = np.random.choice(total_rows, n, replace=False)
    
    # Retrieve the full rows for these random indexes
    random_rows = pandas_df.iloc[random_indexes].values.tolist()
    
    return random_rows

# Initialize an empty list to store all rows
all_random_rows = []

# Get the rows from each DataFrame
for df in tqdm(dfs, desc="Processing DataFrames"):
    rows = get_random_rows(df)
    all_random_rows.extend(rows)

# Print the total number of rows collected
print(len(all_random_rows))


Processing DataFrames: 100%|████████████████████████████████████████████████████████████| 4/4 [17:37<00:00, 264.42s/it]

100000


In [4]:
# Remove the first two items from each element in all_random_rows
new_all_random_rows = [row[2:] for row in all_random_rows]
np.shape(new_all_random_rows)

(100000, 2048)

In [5]:
import pickle
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

# Assuming new_all_random_rows has been populated as described

# Convert new_all_random_rows to a NumPy array
data = np.array(new_all_random_rows)

batch_size = 10000
max_iter = 100

# Apply K-means clustering
kmeans = MiniBatchKMeans(n_clusters=1000, batch_size=batch_size, max_iter=max_iter, random_state=42)
kmeans.fit(data)

# Store the cluster centers
centers = kmeans.cluster_centers_

# Print the shape of the cluster centers
print(f"Shape of the cluster centers: {centers.shape}")

# Save the cluster centers to a pickle file
with open('cluster_centers.pkl', 'wb') as file:
    pickle.dump(centers, file)

print("Cluster centers have been saved to 'cluster_centers.pkl'.")


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


Shape of the cluster centers: (1000, 2048)
Cluster centers have been saved to 'cluster_centers.pkl'.


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)


In [6]:
x_train = X_train
y_train = Y_train

In [7]:
from PIL import Image
import numpy as np
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
import glob

target_size = (32, 32)  # Change the values as per your requirement
# Load the pre-trained ResNet50 model with modified input shape
model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(target_size[0], target_size[1], 3))

ft = model.predict(np.array(x_train).astype("float32"))

from sklearn.cluster import MiniBatchKMeans

n_clusters = 400
batch_size = 100
max_iter = 100

kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, max_iter=max_iter)
kmeans.fit(ft)
# Retrieve the cluster centers
ct = kmeans.cluster_centers_.tolist()

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [9]:
np.shape(ct)

(400, 2048)

In [10]:
import numpy as np
from tqdm import tqdm

# Initialize a list to store the distances
tot_dist = []

# Calculate L2 distances
for i in tqdm(range(len(ct))):
    distances = []
    for j in range(len(centers)):
        distance = np.linalg.norm(ct[i] - centers[j])
        distances.append(distance)
    tot_dist.append(distances)

# Convert the list to a numpy array
tot_dist = np.array(tot_dist)

# Print the shape of the resulting array
print(f"Shape of the distance array: {tot_dist.shape}")

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:48<00:00,  8.26it/s]


Shape of the distance array: (400, 1000)


In [11]:
from ortools.linear_solver import pywraplp
dist_list =  np.transpose(tot_dist, (1, 0))
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:36<00:00, 27.55it/s]


Total cost = 34499.063236818794

Cluster 207 assigned to Class 0. Cost: 76.2538709826466
Cluster 89 assigned to Class 2. Cost: 68.37838189447001
Cluster 138 assigned to Class 3. Cost: 108.39003251129522
Cluster 383 assigned to Class 4. Cost: 85.73153563640906
Cluster 380 assigned to Class 6. Cost: 64.08120703551813
Cluster 353 assigned to Class 7. Cost: 79.65443986155162
Cluster 186 assigned to Class 8. Cost: 106.65387436740725
Cluster 341 assigned to Class 9. Cost: 120.07183631574954
Cluster 264 assigned to Class 10. Cost: 61.80326187536922
Cluster 181 assigned to Class 11. Cost: 83.46243017738098
Cluster 294 assigned to Class 14. Cost: 125.25119369248732
Cluster 224 assigned to Class 15. Cost: 74.39360031401229
Cluster 372 assigned to Class 18. Cost: 72.00200103674577
Cluster 266 assigned to Class 20. Cost: 68.8813390336442
Cluster 392 assigned to Class 22. Cost: 73.77314063347103
Cluster 370 assigned to Class 25. Cost: 71.09834486335471
Cluster 193 assigned to Class 26. Cost: 130.39

Cluster 310 assigned to Class 501. Cost: 71.01298105259106
Cluster 59 assigned to Class 503. Cost: 63.803379247241146
Cluster 359 assigned to Class 504. Cost: 98.88817638845299
Cluster 204 assigned to Class 506. Cost: 75.90018674293437
Cluster 170 assigned to Class 509. Cost: 52.323697294270175
Cluster 43 assigned to Class 515. Cost: 89.41555829996861
Cluster 327 assigned to Class 516. Cost: 129.06720997346855
Cluster 339 assigned to Class 517. Cost: 108.87850240386669
Cluster 363 assigned to Class 518. Cost: 71.19549037647978
Cluster 303 assigned to Class 520. Cost: 83.84148186023478
Cluster 142 assigned to Class 522. Cost: 129.26309555049846
Cluster 253 assigned to Class 523. Cost: 127.73637570331523
Cluster 74 assigned to Class 526. Cost: 63.547483303057696
Cluster 220 assigned to Class 529. Cost: 69.89537883323419
Cluster 34 assigned to Class 533. Cost: 111.16264165354632
Cluster 122 assigned to Class 535. Cost: 81.6892184341848
Cluster 337 assigned to Class 536. Cost: 81.370127455

In [12]:
len(sol_indexes)

400

In [13]:
with open('cvfinal.pkl', 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Indexes have been saved to 'cvfinal.pkl'.")

Indexes have been saved to 'cvfinal.pkl'.


Time taken: 1.6200830936431885 seconds


In [1]:
import dask.dataframe as dd
import numpy as np
from scipy.spatial.distance import cdist
from dask.distributed import Client
import pickle
import time
from tqdm import tqdm
import dask

# Initialize Dask client
client = Client()

# Function to compute closest indexes
def compute_closest_indexes(chunk, centers, index_set):
    result_list = []
    for idx, row in chunk.iterrows():
        features = np.array(row[2:], dtype=float).reshape(1, -1)
        distances = cdist(features, centers, metric='euclidean')
        closest_center_idx = np.argmin(distances)
        if closest_center_idx in index_set:
            result_list.append((closest_center_idx, row[1]))
    return result_list

# Function to process elements of all DataFrames
def process_elements(dfs, centers, index_set, max_len_per_index=500):
    result_list = []
    index_count = {idx: 0 for idx in index_set}
    delayed_results = []

    for df in tqdm(dfs, desc="Processing DataFrames"):
        print(len(result_list))
        # Iterate over chunks of the DataFrame
        for chunk in tqdm(df.to_delayed(), desc="Processing chunks", leave=False):
            # Process each chunk in parallel
            delayed_result = dask.delayed(compute_closest_indexes)(chunk, centers, index_set)
            delayed_results.append(delayed_result)

        # Compute and collect results
        if delayed_results:
            chunk_results = dask.compute(*delayed_results)
            for chunk_result in chunk_results:
                for closest_center_idx, row_1 in chunk_result:
                    if index_count[closest_center_idx] < max_len_per_index:
                        result_list.append(row_1)
                        index_count[closest_center_idx] += 1
                        if index_count[closest_center_idx] >= max_len_per_index:
                            index_set.remove(closest_center_idx)
                        if all(count >= max_len_per_index for count in index_count.values()):
                            return result_list

    return result_list

# Measure the execution time
start_time = time.time()

# Load the pickle files
with open("F:/ML_notebooks/cvfinal.pkl", 'rb') as file:
    index = pickle.load(file)
with open("F:/ML_notebooks/cluster_centers.pkl", 'rb') as file:
    centers = pickle.load(file)

# Convert index to a set for faster lookup
index_set = set(index)

# Define the paths to the CSV files
file_paths = [
    "G:/image_features.csv",
    "G:/image_features1.csv",
    "G:/image_features2.csv",
    "G:/image_features3.csv"
]

# Load the CSV files into Dask DataFrames
dfs = [dd.read_csv(file_path) for file_path in file_paths]

# Call the function to process elements of each DataFrame
result_list = process_elements(dfs, centers, index_set)

# Save the result_list in a pickle file
with open("cifar_images.pkl", "wb") as file:
    pickle.dump(result_list, file)

end_time = time.time()

# Print the execution time
print(f"Execution time: {end_time - start_time} seconds")


Processing DataFrames:   0%|                                                           | 0/4 [00:00<?, ?it/s]

0



Processing chunks:  95%|█████████████████████████████████████████████████▎  | 92/97 [00:00<00:00, 903.52it/s]
                                                                                                             C:\Users\shaif\anaconda3\envs\dasks\lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 16.00 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Processing DataFrames:  25%|████████████▌                                     | 1/4 [12:51<38:35, 771.88s/it]

112774



Processing chunks:   0%|                                                             | 0/101 [00:00<?, ?it/s]
                                                                                                             C:\Users\shaif\anaconda3\envs\dasks\lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 16.35 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Processing DataFrames:  50%|████████████████████████▌                        | 2/4 [42:04<44:57, 1348.99s/it]

187772



Processing chunks:   0%|                                                              | 0/57 [00:00<?, ?it/s]
                                                                                                             C:\Users\shaif\anaconda3\envs\dasks\lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 16.63 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Processing DataFrames:  75%|███████████████████████████████████▎           | 3/4 [1:16:42<28:01, 1681.90s/it]

194458



Processing chunks:   0%|                                                              | 0/43 [00:00<?, ?it/s]
                                                                                                             C:\Users\shaif\anaconda3\envs\dasks\lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 16.90 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Processing DataFrames: 100%|███████████████████████████████████████████████| 4/4 [1:56:28<00:00, 1747.16s/it]

Execution time: 7000.50746011734 seconds


In [2]:
print(len(result_list))

196705
